# API - WEB PROJECT

## 1.- API 

## Retrieve set cards from API

In [65]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import time

In [2]:
complete = []
url = 'https://api.scryfall.com/sets'
response = requests.get(url)
results = response.json()
x = json_normalize(results)
flattened_data1 = json_normalize(x.data[0])
complete.append(flattened_data1)
scryfall = pd.concat(complete, sort=False)

In [3]:
scryfall.head()

,object,id,code,name,uri,scryfall_uri,search_uri,released_at,set_type,card_count,parent_set_code,digital,foil_only,icon_svg_uri,tcgplayer_id,mtgo_code,arena_code,block_code,block
0,set,8fe3f935-7c8d-4a4e-a051-c0b0f251d262,tund,Unsanctioned Tokens,https://api.scryfall.com/sets/8fe3f935-7c8d-4a...,https://scryfall.com/sets/tund,https://api.scryfall.com/cards/search?order=se...,2020-02-29,token,6,und,False,False,https://img.scryfall.com/sets/default.svg?1581...,NaN,NaN,NaN,NaN,NaN
1,set,fccfdf97-f5f2-43b4-9be9-9255414e6633,und,Unsanctioned,https://api.scryfall.com/sets/fccfdf97-f5f2-43...,https://scryfall.com/sets/und,https://api.scryfall.com/cards/search?order=se...,2020-02-29,funny,96,NaN,False,False,https://img.scryfall.com/sets/default.svg?1581...,2598.0,NaN,NaN,NaN,NaN
2,set,66d787e4-101d-4f72-a4ed-7c38df9b99fe,pthb,Theros Beyond Death Promos,https://api.scryfall.com/sets/66d787e4-101d-4f...,https://scryfall.com/sets/pthb,https://api.scryfall.com/cards/search?order=se...,2020-01-24,promo,136,thb,False,False,https://img.scryfall.com/sets/thb.svg?1581310800,NaN,NaN,NaN,NaN,NaN
3,set,200c397b-bf57-46a2-8ebf-592148fd49a4,tthb,Theros Beyond Death Tokens,https://api.scryfall.com/sets/200c397b-bf57-46...,https://scryfall.com/sets/tthb,https://api.scryfall.com/cards/search?order=se...,2020-01-24,token,14,thb,False,False,https://img.scryfall.com/sets/thb.svg?1581310800,NaN,NaN,NaN,NaN,NaN
4,set,5f23a78d-cda1-462a-8be3-a62b40c34913,thb,Theros Beyond Death,https://api.scryfall.com/sets/5f23a78d-cda1-46...,https://scryfall.com/sets/thb,https://api.scryfall.com/cards/search?order=se...,2020-01-24,expansion,358,NaN,False,False,https://img.scryfall.com/sets/thb.svg?1581310800,2568.0,thb,thb,NaN,NaN


In [14]:
#extract the legal core and expansion sets which are from Return to Ravnica to actual
core = scryfall[scryfall['set_type'] == 'core']
expansion = scryfall[scryfall['set_type'] == 'expansion']
set_list = pd.concat([core, expansion])
set_list = set_list.filter(['code', 'name', 'released_at', 'block', 'card_count'])

# the initial sets include from 2012-10-05 to actual set
pioneer_sets = set_list[set_list['released_at'] >= '2012-10-05']

#append 'Welcome Decks' which are also legal
pioneer_sets = pd.concat([pioneer_sets, set_list.filter(like='Welcome Deck', axis=0)])
pioneer_sets = pioneer_sets.sort_values(by=['released_at'])
pioneer_sets['code'] = pioneer_sets['code'].apply(lambda x: x.upper())
pioneer_sets = pioneer_sets.set_index('code',inplace=False).fillna('Unknown')
pioneer_sets.columns = ['Set Name', 'Release Date', 'Block', 'Total Cards']
pioneer_sets['Total Cards'].sum()

,Set Name,Release Date,Block,Total Cards
code,,,,
RTR,Return to Ravnica,2012-10-05,Return to Ravnica,274
GTC,Gatecrash,2013-02-01,Return to Ravnica,249
DGM,Dragon's Maze,2013-05-03,Return to Ravnica,156
M14,Magic 2014,2013-07-19,Core Set,249
THS,Theros,2013-09-27,Theros,249
BNG,Born of the Gods,2014-02-07,Theros,165
JOU,Journey into Nyx,2014-05-02,Theros,165
M15,Magic 2015,2014-07-18,Core Set,284
KTK,Khans of Tarkir,2014-09-26,Khans of Tarkir,269


## Retrieve the list of all cards from the legal sets

In [48]:
# retrieve the number of pages needed for request
total_cards_set = pioneer_sets['Total Cards'].to_list()
page_numbers = [i // 100 + 1 for i in total_cards_set]
#url = 'https://api.magicthegathering.io/v1/cards?page=1&set=RTR'

complete = []
for i in pioneer_sets.index:
    for j in range(1,page_numbers[0]+1):
        url='https://api.magicthegathering.io/v1/cards?page='+str(j)+'&set='+str(i)
        response = requests.get(url)
        time.sleep(0.25)
        results = response.json()
        x = json_normalize(results)
        flattened_data1 = json_normalize(x.cards[0])
        complete.append(flattened_data1)
        
data = pd.concat(complete, sort=False, ignore_index=True)
data

,name,manaCost,cmc,colors,colorIdentity,type,supertypes,types,subtypes,rarity,...,printings,originalText,originalType,legalities,id,power,toughness,names,variations,loyalty
0,Abrupt Decay,{B}{G},2.0,"[Black, Green]","[B, G]",Instant,[],[Instant],[],Rare,...,"[GK1, MM3, PRM, PWCQ, RTR]",Abrupt Decay can't be countered by spells or a...,Instant,"[{'format': 'Commander', 'legality': 'Legal'},...",e14c52f4-73fc-5e68-bf9a-bd066ab860f5,NaN,NaN,NaN,NaN,NaN
1,Aerial Predation,{2}{G},3.0,[Green],[G],Instant,[],[Instant],[],Common,...,"[IMA, RTR]",Destroy target creature with flying. You gain ...,Instant,"[{'format': 'Commander', 'legality': 'Legal'},...",8a867e2f-cca4-53d0-9738-9afcef896f91,NaN,NaN,NaN,NaN,NaN
2,Angel of Serenity,{4}{W}{W}{W},7.0,[White],[W],Creature — Angel,[],[Creature],[Angel],Mythic,...,"[C15, CM2, RTR]",Flying\nWhen Angel of Serenity enters the batt...,Creature — Angel,"[{'format': 'Commander', 'legality': 'Legal'},...",74678642-4fe9-5f54-b24e-eedd0f650963,5,6,NaN,NaN,NaN
3,Annihilating Fire,{1}{R}{R},3.0,[Red],[R],Instant,[],[Instant],[],Common,...,[RTR],Annihilating Fire deals 3 damage to target cre...,Instant,"[{'format': 'Commander', 'legality': 'Legal'},...",7e4357ce-756d-527a-8b96-06ea15d25d0d,NaN,NaN,NaN,NaN,NaN
4,Aquus Steed,{3}{U},4.0,[Blue],[U],Creature — Beast,[],[Creature],[Beast],Uncommon,...,[RTR],"{2}{U}, {T}: Target creature gets -2/-0 until ...",Creature — Beast,"[{'format': 'Commander', 'legality': 'Legal'},...",ffecee7f-39d5-5613-a478-a5f3b743f548,1,3,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7671,Temple of Malice,NaN,0.0,[],"[B, R]",Land,[],[Land],[],Rare,...,"[BNG, THB]",NaN,NaN,"[{'format': 'Commander', 'legality': 'Legal'},...",bb357147-f59a-556c-bd00-0d415fed2633,NaN,NaN,NaN,[fd0b7b81-f859-54b0-912e-c39d77d7802b],NaN
7672,Temple of Malice,NaN,0.0,[],"[B, R]",Land,[],[Land],[],Rare,...,"[BNG, THB]",NaN,NaN,"[{'format': 'Commander', 'legality': 'Legal'},...",fd0b7b81-f859-54b0-912e-c39d77d7802b,NaN,NaN,NaN,[bb357147-f59a-556c-bd00-0d415fed2633],NaN
7673,Temple of Plenty,NaN,0.0,[],"[G, W]",Land,[],[Land],[],Rare,...,"[BNG, THB]",NaN,NaN,"[{'format': 'Commander', 'legality': 'Legal'},...",fae44900-d90d-5466-9c48-92caa222d53c,NaN,NaN,NaN,[5017f256-7ad1-5775-a8e2-bf4c64d9a77f],NaN
7674,Temple of Plenty,NaN,0.0,[],"[G, W]",Land,[],[Land],[],Rare,...,"[BNG, THB]",NaN,NaN,"[{'format': 'Commander', 'legality': 'Legal'},...",5017f256-7ad1-5775-a8e2-bf4c64d9a77f,NaN,NaN,NaN,[fae44900-d90d-5466-9c48-92caa222d53c],NaN


In [50]:
data.columns

Index(['name', 'manaCost', 'cmc', 'colors', 'colorIdentity', 'type',
       'supertypes', 'types', 'subtypes', 'rarity', 'set', 'setName', 'text',
       'flavor', 'artist', 'number', 'layout', 'multiverseid', 'imageUrl',
       'watermark', 'rulings', 'foreignNames', 'printings', 'originalText',
       'originalType', 'legalities', 'id', 'power', 'toughness', 'names',
       'variations', 'loyalty'],
      dtype='object')

In [64]:
cards = data[['name', 'manaCost', 'cmc', 'colors', 'colorIdentity', 'type', 'rarity', 'set', 'text', 'power', 'toughness', 'loyalty']]

cards.type.unique()

# separate subset by card type

array(['Instant', 'Creature — Angel', 'Creature — Beast',
       'Creature — Archon', 'Creature — Spider', 'Creature — Wurm',
       'Creature — Giant Soldier', 'Enchantment — Aura',
       'Creature — Human Warrior', 'Sorcery', 'Creature — Human Druid',
       'Creature — Elk', 'Creature — Human Advisor',
       'Creature — Human Soldier', 'Land — Gate',
       'Creature — Human Wizard', 'Artifact', 'Creature — Lizard',
       'Creature — Weird', 'Land — Swamp Mountain', 'Creature — Giant',
       'Creature — Nightmare Horse', 'Creature — Slug',
       'Creature — Centaur Cleric', 'Creature — Elf Scout',
       'Creature — Imp', 'Artifact — Equipment', 'Creature — Elemental',
       'Enchantment', 'Creature — Pegasus', 'Creature — Fungus',
       'Creature — Vedalken', 'Creature — Horror', 'Creature — Spirit',
       'Creature — Zombie', 'Creature — Insect', 'Creature — Elf Shaman',
       'Creature — Demon', 'Creature — Homunculus', 'Creature — Rat',
       'Creature — Plant Zombie',

## Extract the decklist from the last competitive tournaments

In [67]:
url = 'https://magic.wizards.com/es/game-info/gameplay/formats/pioneer'

def get_links(url):
    html = requests.get(url).content
    soup = BeautifulSoup(html, features='html.parser')
    links = soup.find_all('a', {'class':'autocard-link'})
    # from each link take the 2 second last (which contains the name of the card)
    # and strip the last character that are related with 
    reference = [str(i).split('>')[-2][:-3] for i in links]
    return reference

banned_cards = get_links(url)

In [68]:
print(banned_cards)

['Bloodstained Mire', 'Felidar Guardian', 'Field of the Dead', 'Flooded Strand', 'Leyline of Abundance', 'Nexus of Fate', 'Oath of Nissa', 'Oko, Thief of Crowns', 'Once Upon a Time', 'Polluted Delta', 'Smuggler’s Copter', 'Veil of Summer', 'Windswept Heath ', 'Wooded Foothills']


In [155]:
url = 'https://magic.wizards.com/en/events/coverage/players-tour-brussels-2020-decklists-1'
html = requests.get(url).content
soup = BeautifulSoup(html, features='html.parser')
titles = soup.find_all('h4')

card_list = []
card_count = []
span_cards = []
for deck in soup.find_all('div', {'class':'deck-list-text'}):
    a_tag = deck.find_all('span', {'class':'card-name'})
    count_tag = deck.find_all('span', {'class':'card-count'})
    name = [str(tag).split('>')[-3][:-3] for tag in a_tag]
    count = [tag.text for tag in count_tag]
    # change non cards 'span class'
    span = [str(tag).split('>')[-2][:-6] for tag in a_tag]
    
    #append the results
    card_list.append(name)
    card_count.append(count)
    span_cards.append(span)

for i, deck in enumerate(card_list):
    for j, card in enumerate(deck):
        if '<span class' in card:
            card_list[i][j] = span_cards[i][j]
        


In [204]:
# retrieve the decks from Brussels Tournament

card_list = []
card_count = []
span_cards = []
player_deck = []

for s in range(1,6):
    url = 'https://magic.wizards.com/en/events/coverage/players-tour-brussels-2020-decklists-'+str(s)
    html = requests.get(url).content
    soup = BeautifulSoup(html, features='html.parser')
    player = [str(title.text) for title in soup.find_all('h4')]
    # append the results
    player_deck.append(player)
    
    for deck in soup.find_all('div', {'class':'deck-list-text'}):
        a_tag = deck.find_all('span', {'class':'card-name'})
        count_tag = deck.find_all('span', {'class':'card-count'})
        name = [str(tag).split('>')[-3][:-3] for tag in a_tag]
        count = [tag.text for tag in count_tag]
        
        # change non cards 'span class'
        span = [str(tag).split('>')[-2][:-6] for tag in a_tag]
    
        #append the results
        card_list.append(name)
        card_count.append(count)
        span_cards.append(span)

for i, deck in enumerate(card_list):
    for j, card in enumerate(deck):
        if '<span class' in card:
            card_list[i][j] = span_cards[i][j]
            
player_list = [i.split(',')[0] + str(',') + i.split(',')[1].split('-')[0] for page in player_deck for i in page]
deck_list = ['-'.join(i.split(',')[1].split('-')[1:]) for page in player_deck for i in page]

In [241]:
#player_list = [i.split('-')[0] for page in player_deck for i in page]
#deck_list = ['-'.join(i.split('-')[1:]) for page in player_deck for i in page]

player_list

['Adrien, Houssard ',
 'Airaksinen, Mikko ',
 'AlAwadhi, Abdullah ',
 'Albertini, Leonardo ',
 'Alberyd, Johannes ',
 'Alcaraz Coca, Aniol ',
 'Almela Torres, Amadeo ',
 'Alsabah, Mohammad ',
 'Alvarez, Adrian ',
 'Amendola, Michele ',
 'Andren, Lukas ',
 'Andrzejewski, Alan ',
 'Anteri, Fabrizio ',
 'Antunovic, Dado ',
 'AUREJAC, Alexandre ',
 'Avgoustis, Kyriakos ',
 'Avila Abadia, Carlos ',
 'Balfour, Brandon ',
 'Ballester Garcia, Carlos ',
 'Bastogne, Jerome ',
 'Bausch, Joshua ',
 'Begon, Ludovic ',
 'Behal, Reamonn ',
 'Benias, Nikolas ',
 'benincasa, giuliano ',
 'BERTEAUX, Julien ',
 'Bezruchenko, Dmitryi ',
 'BIASIO, STEFANO ',
 'Bitzberger, Eric ',
 'Bleriot, Aurelien ',
 'Blohon, Lukas ',
 'Bonafonte, Guillem ',
 'Bonde, Michael ',
 'Borabo, Erickson ',
 'Bornoz, Tristan ',
 'Boussaud, Eliott ',
 'Brown, Matt ',
 'Broxup, Ross ',
 'Buchan, Joshua ',
 'Budde, Kai ',
 'Burchett, Autumn ',
 'Busson, Etienne ',
 'Cabezas, Jose ',
 'Cabrer, Marc ',
 'Cai, Giona ',
 'Caillaba, Ge

In [244]:
brussels = pd.DataFrame([player_list, deck_list, card_count, card_list], index=['Player', 'Deck', 'Copies', 'Cards'])
brussels.T

,Player,Deck,Copies,Cards
0,"Adrien, Houssard",Mono-Black Aggro,"[4, 4, 4, 4, 3, 4, 1, 1, 4, 2, 4, 1, 4, 4, 16,...","[Bloodsoaked Champion, Dread Wanderer, Knight ..."
1,"Airaksinen, Mikko",Dimir Inverter,"[2, 4, 4, 2, 2, 2, 2, 4, 4, 4, 1, 4, 1, 1, 4, ...","[Jace, Wielder of Mysteries, Inverter of Truth..."
2,"AlAwadhi, Abdullah",Dimir Inverter,"[3, 4, 4, 2, 4, 2, 4, 2, 4, 4, 2, 2, 4, 4, 2, ...","[Jace, Wielder of Mysteries, Inverter of Truth..."
3,"Albertini, Leonardo",Dimir Inverter,"[2, 4, 4, 1, 4, 3, 4, 4, 2, 1, 2, 4, 2, 4, 3, ...","[Jace, Wielder of Mysteries, Inverter of Truth..."
4,"Alberyd, Johannes",Big Red,"[2, 4, 3, 4, 4, 2, 4, 2, 4, 4, 2, 2, 15, 4, 4,...","[Chandra, Torch of Defiance, Bonecrusher Giant..."
...,...,...,...,...
379,"Worsnop, George",Niv to Light,"[1, 4, 1, 4, 2, 1, 4, 1, 1, 3, 4, 1, 1, 1, 1, ...","[Nahiri, the Harbinger, Teferi, Time Raveler, ..."
380,"wright, stuart",Simic Ramp,"[3, 4, 4, 4, 1, 2, 4, 2, 4, 4, 3, 4, 4, 3, 8, ...","[Ugin, the Spirit Dragon, Arboreal Grazer, Cav..."
381,"Wu Martin, Eneko",Dimir Control,"[2, 4, 1, 2, 1, 4, 1, 2, 4, 4, 4, 4, 4, 4, 4, ...","[Liliana, the Last Hope, Jace, Vryn's Prodigy,..."
382,"Zaccarini, Gabriele",Azorius Control,"[1, 2, 2, 3, 2, 4, 4, 4, 2, 4, 2, 2, 2, 1, 2, ...","[Gideon of the Trials, Jace, Architect of Thou..."


In [246]:
# retrieve the decks from Nagoya Tournament

card_list = []
card_count = []
span_cards = []
player_deck = []

for s in range(1,4):
    url = 'https://magic.wizards.com/en/events/coverage/players-tour-nagoya-2020-decklists-'+str(s)
    html = requests.get(url).content
    soup = BeautifulSoup(html, features='html.parser')
    player = [str(title.text) for title in soup.find_all('h4')]
    # append the results
    player_deck.append(player)
    
    for deck in soup.find_all('div', {'class':'deck-list-text'}):
        a_tag = deck.find_all('span', {'class':'card-name'})
        count_tag = deck.find_all('span', {'class':'card-count'})
        name = [str(tag).split('>')[-3][:-3] for tag in a_tag]
        count = [tag.text for tag in count_tag]
        
        # change non cards 'span class'
        span = [str(tag).split('>')[-2][:-6] for tag in a_tag]
    
        #append the results
        card_list.append(name)
        card_count.append(count)
        span_cards.append(span)

for i, deck in enumerate(card_list):
    for j, card in enumerate(deck):
        if '<span class' in card:
            card_list[i][j] = span_cards[i][j]
            
player_list = [i.split(',')[0] + str(',') + i.split(',')[1].split('-')[0] for page in player_deck for i in page]
deck_list = ['-'.join(i.split(',')[1].split('-')[1:]) for page in player_deck for i in page]

In [324]:
nagoya = pd.DataFrame([player_list, deck_list, card_count, card_list], index=['Player', 'Deck', 'Copies', 'Cards'])
nagoya = nagoya.T
nagoya

,Player,Deck,Copies,Cards
0,"Anan, Kazuyoshi",Niv to Light,"[4, 1, 1, 3, 2, 4, 4, 2, 4, 1, 1, 1, 1, 1, 2, ...","[Teferi, Time Raveler, Hostage Taker, Kunoros,..."
1,"Asahara, Akira",Blue Black Inverter,"[3, 1, 4, 4, 4, 4, 4, 4, 4, 3, 5, 2, 1, 1, 4, ...","[Jace, Wielder of Mysteries, Liliana, the Last..."
2,"Asano, Kurenai",Mono Black Aggro,"[4, 4, 4, 4, 3, 3, 4, 4, 1, 4, 1, 15, 4, 4, 1,...","[Bloodsoaked Champion, Dread Wanderer, Knight ..."
3,"B.abella, Lexter",Big Red,"[4, 4, 4, 2, 4, 4, 4, 4, 2, 4, 17, 2, 1, 4, 1,...","[Abbot of Keral Keep, Bonecrusher Giant, Gobli..."
4,"Bai, Hualin",Blue White Control,"[1, 1, 2, 3, 1, 4, 2, 4, 2, 2, 1, 4, 2, 2, 1, ...","[Elspeth, Sun's Champion, Narset, Parter of Ve..."
...,...,...,...,...
186,"Yasooka, Shota",Blue Black Inverter,"[1, 3, 4, 2, 1, 1, 1, 4, 3, 3, 1, 4, 1, 1, 4, ...","[Ashiok, Nightmare Muse, Jace, Wielder of Myst..."
187,"Yazu, Yasuhiro",Big Red,"[1, 4, 4, 4, 3, 4, 4, 4, 4, 4, 17, 2, 1, 4, 1,...","[Chandra, Torch of Defiance, Abbot of Keral Ke..."
188,"Yimin, Zhi",Niv to Light,"[1, 4, 1, 3, 2, 1, 4, 1, 3, 4, 1, 1, 1, 2, 1, ...","[Nahiri, the Harbinger, Teferi, Time Raveler, ..."
189,"Yukuhiro, Ken",Sram Auras,"[2, 4, 4, 3, 4, 4, 4, 4, 4, 3, 4, 6, 1, 4, 4, ...","[Favored Hoplite, Sram, Senior Edificer, Alsei..."


In [290]:
## Retrieve the results from Brussels Matchs

url = 'https://magic.gg/news/players-tour-brussels-2020-round-1-results'
html = requests.get(url).content
soup = BeautifulSoup(html, features='html.parser')
table = soup.find_all('div', {'class': 'table-wrapper'})[0].find_all('tr')

tabs = []
for tr in table:
        tabs.append({
            'Player': tr.find_all('td')[1].string,
            'Opponent': tr.find_all('td')[5].string,
            'Result': tr.find_all('td')[3].string
            })
round1 = pd.DataFrame(tabs)
round1

,Player,Opponent,Result
0,"Caparroz, Joao Lucas","Gauthier, Martin-Eric",Won 2-0
1,"Knittel, Fabrice","Calafell, Joel",Won 2-0
2,"Oneto, Mattia","Choca, Joao",Lost 0-2
3,"Jao, Jeff","Pereira, Alexandre",Won 2-1
4,"Wijsman, Gerwin","Rodrigues, Thiago",Lost 1-2
...,...,...,...
187,"Perbet, Guillaume","Kuzmenko, Nazar",Lost 1-2
188,"Pinto, Goncalo","Inman, Jesse",Lost 0-2
189,"Thaler, Sebastian","Llamas Nuflo, Diego",Won 2-1
190,"Holzinger, Thomas","Gaieski, Ethan",Lost 1-2


In [322]:
## Retrieve the results from Brussels Matchs

tabs = []
for i in range(1,17):
    url = 'https://magic.gg/news/players-tour-brussels-2020-round-'+str(i)+'-results'
    html = requests.get(url).content
    soup = BeautifulSoup(html, features='html.parser')
    table = soup.find_all('div', {'class': 'table-wrapper'})[0].find_all('tr')
    
    for tr in table:
        tabs.append({
            'Player': tr.find_all('td')[1].string,
            'Opponent': tr.find_all('td')[5].string,
            'Match': tr.find_all('td')[3].string
            })

brussels_round = pd.DataFrame(tabs)

#transform result table into classification
brussels_round['Result'] = brussels_round['Match'].str.split('\s').str.get(0)
brussels_round['GW'] = brussels_round['Match'].str.split('\s').str.get(1).str.split('-').str.get(0)
brussels_round['GL'] = brussels_round['Match'].str.split('\s').str.get(1).str.split('-').str.get(1)

brussels_round['Player'].unique()

array(['Caparroz, Joao Lucas ', 'Knittel, Fabrice ', 'Oneto, Mattia ',
       'Jao, Jeff ', 'Wijsman, Gerwin ', 'Liu, Jin ', 'Strasky, Ondrej ',
       'Vinci, Stefano ', 'Signargout, Axel ', 'Alsabah, Mohammad ',
       'Hendriks, Thomas ', 'Roca, Caua ', 'Silva, Marco ',
       'Kowalski, Grzegorz ', 'Patarca, Humberto ', 'Calf, David ',
       'Chapman, Craig ', 'Matignon, Guillaume ', 'Silcock, Ross ',
       'De Jesus Alves Pereira, Gabriel ', 'Giorgio, Francesco ',
       'Cabrer, Marc ', 'Reina Rabal, Carlos ', 'Villa Perez, Pablo ',
       'Prinz, Kristof ', 'Kats, Mikhail ', 'Kwasniewski, Artur ',
       'Grafensteiner, Tobias ', 'Dezani, Jeremy ', 'White, Paul ',
       'Rizzi, Mattia ', 'Grancha, Beatriz ', 'Caputi, Simone ',
       'Esper Berthoud, Lucas ', 'Martin Jimenez, Luis ',
       'Chabriac, Nicolas ', 'Kuisma, Matti ', 'Torres, Amadeo ',
       'Trotte, Florian ', 'Hyttel, Lucas ', 'Cestaro, Laurent ',
       'Balfour, Brandon ', 'Wright, Stuart ', 'Semenov, Petr '

In [346]:
## Retrieve the results from Nagoya Matchs

tabs = []
for i in range(1,16):
    url = 'https://magic.gg/news/players-tour-nagoya-2020-round-'+str(i)+'-results'
    html = requests.get(url).content
    soup = BeautifulSoup(html, features='html.parser')
    table = soup.find_all('div', {'class': 'table-wrapper'})[0].find_all('tr')
    
    for tr in table:
        tabs.append({
            'Player': tr.find_all('td')[1].string,
            'Opponent': tr.find_all('td')[5].string,
            'Match': tr.find_all('td')[3].string
            })

nagoya_round = pd.DataFrame(tabs)

#transform result table into classification
nagoya_round['Result'] = nagoya_round['Match'].str.split('\s').str.get(0)
nagoya_round['GW'] = nagoya_round['Match'].str.split('\s').str.get(1).str.split('-').str.get(0)
nagoya_round['GL'] = nagoya_round['Match'].str.split('\s').str.get(1).str.split('-').str.get(1)

nagoya_round.Player.value_counts()

Kumagai, Riku          13
Nan, Tu                12
Calcano, Christian     12
Butakov, Dmitriy       11
Harane, Kenta          10
                       ..
Anan, Kazuyoshi         1
Nicholson, Matthew      1
Segou, Kenji            1
Boku, Takashi           1
Nishiura, Ryosuke       1
Name: Player, Length: 190, dtype: int64

In [344]:
result = pd.merge(nagoya_round, nagoya, how='left', on=['Player'])
test = result[['Deck', 'Opponent', 'Result', 'GW', 'GL']]
test.columns = ['Player Deck', 'Player', 'Result', 'GW', 'GL']
test

,Player Deck,Player,Result,GW,GL
0,Lotus Breach Combo,"Nishimoto, Sosuke",Won,2,1
1,Gb Stompy,"Masahide, Moriyama",Won,2,0
2,Humans,"Lip, Quinton",Lost,1,2
3,NaN,"Sze Hang, Chan",Lost,1,2
4,Izzet Ensoul,"Boku, Takashi",Lost,1,2
...,...,...,...,...,...
937,Mono Red Aggro,"Abella, Lexter",Lost,0,2
938,Big Red,"Nakamura, Shuhei",Won,2,1
939,NaN,"Takahashi, Eiji",Lost,1,2
940,NaN,"Garces, Craig Shaun",Won,2,0
